**LOAD LIBRARIES**

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
import json
import tweepy as tp
import twitter_credentials
import io
import requests

# 1.0 DATA COLLECTION - BOT READER

This Notebook reads Tweets from known bots that have been publishded to nbcnews.com.

https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731

**READ DATA INTO DATAFRAMES FROM FROM NBC Website**



In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 14 22:45:11 2018
@authors: andrewcaide and eumarassis
bot list pulled from:
    https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731
"""


tweets_url = "http://nodeassets.nbcnews.com/russian-twitter-trolls/tweets.csv"
tweets_content = requests.get(tweets_url).content

bots_url = "http://nodeassets.nbcnews.com/russian-twitter-trolls/users.csv"
bots_content = requests.get(bots_url).content

bots = pd.read_csv(io.StringIO(bots_content.decode('utf-8')))
tweets = pd.read_csv(io.StringIO(tweets_content.decode('utf-8')))

print(tweets.dtypes)

def produce_bot_LoDs(bots, tweets):
    '''
    Read in dataframe of bots and their tweets. Organize them according to Mark's format. 
    
    Args: 
        dataframes - Bots, Tweets
    
    Returns: 
        Cleaned dataframe, with an extra column: 'Bot_Status' = True
    '''
    # Fix tweets:
    tweet_colnames = ['created_at','favorite_count','id','id_str','is_tweet','lang','retweet_count',
                 'source','text','truncated','user_screen_name']
    
    # Ask mark to see if he can pull out hashtags from his tweets!!
    tweets['truncated'] = False
    tweets['is_tweet'] = tweets['text'].apply(lambda x: False if str(x).find('RT') == -1 else True)
    tweets = tweets.rename(columns={'tweet_id': 'id'}) 
    tweets['id_str'] = str(tweets['id'])
    tweets['user_screen_name'] = tweets['user_key']
    tweets['lang'] = 'en'
    #tweets['created_at'] = tweets['created_str']
    
    tweet_output = tweets[tweet_colnames]
    
    # Fix users:
    user_colnames = ['created_at','description','favourites_count','followers_count','id',
                      'lang','listed_count','location','name','screen_name','statuses_count',
                      'time_zone','verified']
    
    bots_output = bots[user_colnames]
    bots_output['known_bot'] = True
    return(bots_output, tweet_output)


bots_Clean, tweets_Clean = produce_bot_LoDs(bots, tweets)

print(bots_Clean.shape)

user_id                  float64
user_key                  object
created_at               float64
created_str               object
retweet_count            float64
retweeted                 object
favorite_count           float64
text                      object
tweet_id                 float64
source                    object
hashtags                  object
expanded_urls             object
posted                    object
mentions                  object
retweeted_status_id      float64
in_reply_to_status_id    float64
dtype: object
(453, 14)


**WRITE DATAFRAMES WITH BOT TWEETS AND USERS TO JSON FILE**

In [8]:
user_json = bots_Clean.to_json(orient='records')
tweet_json = tweets_Clean.to_json(orient='records')

with open('data/b_tweet_table_out.json', 'w') as outfile:  
    json.dump(tweet_json, outfile)
    
with open('data/b_user_table_out.json', 'w') as outfile:  
    json.dump(user_json, outfile)